# Lectura de archivos

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id = '1FAjcexe-71nGuYIzvnQ46IdXVcqM9cx4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values_complete_features.csv')

test_values1 = pd.read_csv('test_values_complete_features.csv', encoding='latin-1', index_col='building_id')

test_values1[test_values1.select_dtypes('O').columns] = test_values1[test_values1.select_dtypes('O').columns].astype('category')

In [4]:
id = '1qs2mEnkqiAqebJE2SvqkrfoV66Edguwr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values_complete_features.csv')

train_values1 = pd.read_csv('train_values_complete_features.csv', encoding='latin-1', index_col='building_id')

train_values1[train_values1.select_dtypes('O').columns] = train_values1[train_values1.select_dtypes('O').columns].astype('category')

In [5]:
id='1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')
train_labels = pd.read_csv('train_labels.csv', encoding='latin-1', dtype={'building_id': 'int64', 'damage_grade': 'int64'}, index_col='building_id')

In [6]:
id='1br3fMwXX_J0XmiXvOm_wfKWvHSj45T3y'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values.csv')
train_values2 = pd.read_csv('train_values.csv', encoding='latin-1', index_col='building_id')

train_values2[train_values2.select_dtypes('O').columns] = train_values2[train_values2.select_dtypes('O').columns].astype('category')

In [7]:
id = '1kt2VFhgpfRS72wtBOBy1KDat9LanfMZU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values.csv')
test_values2 = pd.read_csv('test_values.csv', encoding='latin-1', index_col='building_id')

test_values2[test_values2.select_dtypes('O').columns] = test_values2[test_values2.select_dtypes('O').columns].astype('category')

In [8]:
train_values_complete = train_values1.copy()
test_values_complete = test_values1.copy()
train_values_incomplete = train_values2.copy()
test_values_incomplete = test_values2.copy()

# Logistic Regression

In [35]:
train_values = train_values_incomplete.copy()
test_values = test_values_incomplete.copy()

In [51]:
df = train_values.merge(train_labels, left_index = True, right_index= True)

from sklearn.utils import shuffle
df = shuffle(df)

In [52]:
# 148259, 25124, 87218

In [53]:
df_damage_2 = df.loc[df['damage_grade'] == 2]
df_damage_2 = df_damage_2[:25124]

df_damage_1 = df.loc[df['damage_grade'] == 1]
df_damage_1 = df_damage_1[:25124]

df_damage_3 = df.loc[df['damage_grade'] == 3]
df_damage_3 = df_damage_3[:25124]

In [54]:
df = df_damage_1.append(df_damage_2).append(df_damage_3)
df['damage_grade'].value_counts()

3    25124
2    25124
1    25124
Name: damage_grade, dtype: int64

In [56]:
train_values = df.drop(columns='damage_grade')
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
111048,20,158,621,2,10,17,5,n,w,q,v,x,t,d,0,0,0,0,0,1,1,0,0,0,0,v,1,1,0,1,0,0,0,0,0,0,0,0
603254,26,1401,418,1,15,6,4,t,w,q,f,j,s,d,0,0,0,0,0,0,1,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
121877,26,886,12157,1,15,4,4,n,w,n,f,j,s,d,0,0,0,0,0,0,1,0,0,0,0,v,0,0,0,0,0,0,0,0,0,0,0,0
884785,6,673,2803,2,20,17,7,t,i,x,v,s,t,d,0,0,0,0,0,0,0,0,1,0,0,v,0,1,0,1,0,0,0,0,0,0,0,0
182923,8,463,5839,4,15,61,13,t,i,x,v,s,j,d,0,0,0,0,0,0,0,0,0,1,0,v,1,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800456,6,1289,3103,2,15,4,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
356769,22,1152,8676,2,5,5,4,t,r,n,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1026314,7,1040,791,2,200,4,4,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [23]:
cat_features = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'land_surface_condition', 'foundation_type', 'roof_type', 
                    'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']
                    
num_features = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']

In [24]:
idx = train_values.shape[0]
data_df = pd.concat([train_values, test_values], sort=False)

data_cat = pd.DataFrame(index = data_df.index, 
                  data = data_df, 
                  columns = cat_features)

data_num = data_df.drop(columns = cat_features)

In [25]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop = 'first')
enc.fit(data_cat)

OneHotEncoder(categories='auto', drop='first', dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [26]:
data_cat_encoded = enc.transform(data_cat)

In [27]:
type(data_cat_encoded)

scipy.sparse.csr.csr_matrix

In [28]:
type(data_num)

pandas.core.frame.DataFrame

In [29]:
from scipy.sparse import coo_matrix, hstack
data = hstack((data_cat_encoded,data_num))

In [30]:
data = data.astype(dtype='float16')
X_train = data.tocsr()[:idx]
X_test = data.tocsr()[idx:]

In [31]:
y_train = train_labels['damage_grade']

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [33]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C=1.0, random_state=42, solver='liblinear', max_iter=500)
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
from sklearn.metrics import f1_score

y_pred = log_reg.predict(X_valid)
f1_score(y_valid, y_pred, average='micro')

0.7392413806335258

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
predicted_df = pd.DataFrame(y_pred.astype(np.int8), index = test_values.index, columns=['damage_grade'])
predicted_df.to_csv('submit_log_reg.csv')